In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [2]:
!python -V

Python 3.10.13


In [7]:
import pickle
import pandas as pd
import numpy as np

In [5]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/opt/homebrew/Caskroom/miniforge/base/envs/env_tensorflow/lib/python3.9/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/env_tensorflow/lib/python3.9/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [2]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [3]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [6]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [8]:
print(np.std(y_pred))

6.247488852238703


In [11]:
print(type(y_pred))

<class 'numpy.ndarray'>


In [10]:
year = 2023
month = 3

In [14]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
pred_series = pd.Series(y_pred,name='duration')
df_results = pd.concat([df[['ride_id']],pred_series],axis=1)

In [16]:
df_results.to_parquet(
    'predictions.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)